In [10]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn import preprocessing 
from collections import namedtuple

In [5]:
#loading in the df
df = pd.read_csv("data\\2011-2020.csv")

In [6]:
#filtering df
df = df.dropna()

In [8]:

correlationdf = df[['LATITUDE', 'LONGITUDE', 'STNELEV', 'Year', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
'September', 'October', 'November', 'December']]
correlationdf.corr(method="pearson")


,LATITUDE,LONGITUDE,STNELEV,Year,January,February,March,April,May,June,July,August,September,October,November,December
LATITUDE,1.000000,-0.320991,0.037920,-0.019059,-0.716844,-0.686624,-0.623176,-0.479858,-0.223185,0.051941,0.206645,0.126519,-0.144698,-0.451302,-0.622778,-0.686914
LONGITUDE,-0.320991,1.000000,-0.287447,-0.027336,0.165444,0.173891,0.164519,0.177534,0.086269,-0.055287,-0.145025,-0.125189,-0.044351,0.104209,0.172963,0.187423
STNELEV,0.037920,-0.287447,1.000000,0.022515,-0.167810,-0.187112,-0.200427,-0.285772,-0.342213,-0.268384,-0.216726,-0.223027,-0.266419,-0.289010,-0.245495,-0.246206
Year,-0.019059,-0.027336,0.022515,1.000000,0.043880,0.046035,0.022776,0.005468,0.015052,0.018183,0.011825,0.013516,0.023621,0.001448,0.020356,0.024809
January,-0.716844,0.165444,-0.167810,0.043880,1.000000,0.963475,0.929647,0.830398,0.625759,0.378778,0.250314,0.354972,0.608065,0.843287,0.942169,0.958706
February,-0.686624,0.173891,-0.187112,0.046035,0.963475,1.000000,0.950733,0.863532,0.679036,0.452504,0.318555,0.414540,0.654099,0.868245,0.937023,0.936606
March,-0.623176,0.164519,-0.200427,0.022776,0.929647,0.950733,1.000000,0.932281,0.787119,0.593486,0.470254,0.550610,0.749362,0.907586,0.942281,0.915909
April,-0.479858,0.177534,-0.285772,0.005468,0.830398,0.863532,0.932281,1.000000,0.909855,0.762474,0.645423,0.708354,0.859967,0.940244,0.898842,0.845635
May,-0.223185,0.086269,-0.342213,0.015052,0.625759,0.679036,0.787119,0.909855,1.000000,0.911600,0.829033,0.862214,0.923454,0.879310,0.747293,0.670073
June,0.051941,-0.055287,-0.268384,0.018183,0.378778,0.452504,0.593486,0.762474,0.911600,1.000000,0.947964,0.946915,0.908608,0.748052,0.542973,0.424373


In [ ]:
#we're going to have to create a model for each month due to the nature of the dataset

#fancy tuple for output of create_train_test function
ModelSets = namedtuple('testsets', ["X_train","y_train","X_test","y_test"])

#this function splits the train and test set depending on the month we're creating the model for
def create_train_test(train: pd.DataFrame, test: pd.DataFrame, month: str) -> ModelSets: 
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
    'September', 'October', 'November', 'December']
    X_train = train.drop(columns=months)
    y_train = train[month]

    X_test = test.drop(columns=months)
    y_test = test[month]


    outset = ModelSets(X_train,y_train,X_test,y_test)

    return outset


#creates a model specfic to the month
def create_model(df: pd.DataFrame, month: str):
    percentfortrain = .9
    train = df.sample(frac=percentfortrain)
    test = df.drop(index=train.index)

    modelsets = create_train_test(train,test,month)
    
    lr = LinearRegression(fit_intercept=False)

    lr.fit(modelsets.X_train, modelsets.y_train)
    print("LinearRegression model for " + month)
    print("Score for training")
    print(lr.score(modelsets.X_train, modelsets.y_train))
    print("Score for testing")
    print(lr.score(modelsets.X_test, modelsets.y_test))

    return lr


In [16]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
    'September', 'October', 'November', 'December']

lrmodels = {}

for i in months:
    lrmodels[i] = create_model(correlationdf, i)

LinearRegression model for January
Score for training
0.5468548244507623
Score for testing
0.5461386981626009
LinearRegression model for February
Score for training
0.5056608855296478
Score for testing
0.5251889836862599
LinearRegression model for March
Score for training
0.42817825027006595
Score for testing
0.4299028119867013
LinearRegression model for April
Score for training
0.3051591402343703
Score for testing
0.30380778624085725
LinearRegression model for May
Score for training
0.17071554023489766
Score for testing
0.15162544168683467
LinearRegression model for June
Score for training
0.09198741508566888
Score for testing
0.08846530417515674
LinearRegression model for July
Score for training
0.1174944136795486
Score for testing
0.11647075456122391
LinearRegression model for August
Score for training
0.0956481601266268
Score for testing
0.08409631817865171
LinearRegression model for September
Score for training
0.12125791113806672
Score for testing
0.12399284685571554
LinearRegres